In [ ]:
%matplotlib widget
from processscheduler import *

### Resource assignement
Two tasks, two resources

In [ ]:
# problem
pb_ra = SchedulingProblem(name="ResourceAssignment")
# tasks
t1 = FixedDurationTask(name="t1", duration=2)
t2 = FixedDurationTask(name="t2", duration=2)
t3 = FixedDurationTask(name="t3", duration=2)

# resource requirements
w1 = Worker(name="W1")
w2 = Worker(name="W2")

# resource assignement
t1.add_required_resource(w1)  # t1 only needs w1
t2.add_required_resource(w2)  # t2 only needs w2
t3.add_required_resource(w1)  # t2 needs both w1 AND w2
t3.add_required_resource(w2)

# solve
solver1 = SchedulingSolver(problem=pb_ra)
solution1 = solver1.solve()
solution1.render_gantt_matplotlib()

### TaskDontOverlap constraint
Make two tasks don't overlap, one is scheduled after the other, or the opposite.

In [ ]:
# problem
pb_ovlap = SchedulingProblem(name="DontOverlapExample", horizon=6)
# tasks
t1 = FixedDurationTask(name="t1", duration=2)
t2 = FixedDurationTask(name="t2", duration=2)
t3 = FixedDurationTask(name="t3", duration=2)

# constraints
TaskStartAt(task=t2, value=1)
TasksDontOverlap(task_1=t2, task_2=t3)  # t2 and t3 must not overlap

# solve
solver1 = SchedulingSolver(problem=pb_ovlap)
sol = solver1.solve()
# render_mode set to Tasks because there is no resource
sol.render_gantt_matplotlib(render_mode="Task")

### SelectWorkers assignment
Choose $n$ workers among a list of available workers

In [ ]:
# problem
pb_alt = SchedulingProblem(name="AlternativeWorkerExample")

# tasks
t1 = FixedDurationTask(name="t1", duration=3)
t2 = FixedDurationTask(name="t2", duration=2)
t3 = FixedDurationTask(name="t3", duration=2)

# resource requirements
w1 = Worker(name="W1")
w2 = Worker(name="W2")
w3 = Worker(name="W3")
w4 = SelectWorkers(list_of_workers=[w1, w2, w3], nb_workers_to_select=1, kind="max")

# resource assignement
t1.add_required_resource(w1)  # t1 only needs w1
t2.add_required_resource(w2)  # t2 only needs w2
t3.add_required_resource(w4)  # t3 only needs one of w1, 2 or 3

# solve
solver1 = SchedulingSolver(problem=pb_alt)
sol = solver1.solve()
sol.render_gantt_matplotlib(render_mode="Task")

In [ ]:
pb_alt = SchedulingProblem(name="AlternativeWorkerExample")

# tasks
t1 = FixedDurationTask(name="t1", duration=3)
t2 = FixedDurationTask(name="t2", duration=2)
t3 = FixedDurationTask(name="t3", duration=2)
t4 = FixedDurationTask(name="t4", duration=2)
t5 = FixedDurationTask(name="t5", duration=2)

# resource requirements
w1 = Worker(name="W1")
w2 = Worker(name="W2")
w3 = Worker(name="W3")
w4 = SelectWorkers(list_of_workers=[w1, w2, w3], nb_workers_to_select=1, kind="exact")
w5 = SelectWorkers(list_of_workers=[w1, w2, w3], nb_workers_to_select=2, kind="max")
w6 = SelectWorkers(list_of_workers=[w1, w2, w3], nb_workers_to_select=3, kind="min")

# resource assignement
t1.add_required_resource(w1)  # t1 only needs w1
t2.add_required_resource(w2)  # t2 only needs w2
t3.add_required_resource(w4)  # t3 needs one of w1, 2 or 3
t4.add_required_resource(w5)  # t4 needs at most 2 of w1, w2 or 3
t5.add_required_resource(w6)  # t5 needs at least 3 of w1, w2 or w3

pb_alt.add_objective_makespan()

# solve
solver1 = SchedulingSolver(problem=pb_alt)
sol = solver1.solve()

sol.render_gantt_matplotlib(render_mode="Task")

### Boolean Not for a task constraint

In [ ]:
# problem
pb_not = SchedulingProblem(name="NotOperator", horizon=4)

# tasks
t2 = FixedDurationTask(name="t2", duration=2)

# constraints
not_cstr1 = not_(TaskStartAt(task=t2, value=0))
not_cstr2 = not_(TaskStartAt(task=t2, value=1))

pb_not.add_constraint(not_cstr1)
pb_not.add_constraint(not_cstr2)

# solve
solver = SchedulingSolver(problem=pb_not)
sol = solver.solve()
print(sol.to_json_string())
sol.render_gantt_matplotlib()

### Boolean Or between tasks constraints

In [ ]:
# problem
pb_or = SchedulingProblem(name="OrOperator", horizon=7)

# tasks
task = FixedDurationTask(name="task", duration=2)

# constraint
or_cstr = or_([TaskStartAt(task=task, value=3), TaskStartAt(task=task, value=2)])
pb_or.add_constraint(or_cstr)

# solve
solver = SchedulingSolver(problem=pb_or)
sol = solver.solve()
sol.render_gantt_matplotlib()

### Boolean And between tasks constraints

In [ ]:
# problem
pb_and = SchedulingProblem(name="AndOperator", horizon=8)

# tasks
task = FixedDurationTask(name="task", duration=2)

# constraint
and_cstr = and_([TaskStartAfter(task=task, value=1), TaskEndBefore(task=task, value=4)])
pb_and.add_constraint(and_cstr)

# solve
solver = SchedulingSolver(problem=pb_and)
sol = solver.solve()
sol.render_gantt_matplotlib()

In [ ]:
problem = SchedulingProblem(name="FindAnotherSolution", horizon=6)
nb_solution = 0
# only one task, there are many diffrent solutions
task_1 = FixedDurationTask(name="task1", duration=2)

solver = SchedulingSolver(problem=problem)
solution = solver.solve()

assert solution

while solution:
    nb_solution += 1
    print("Solution found for task1 start:", solution.tasks[task_1.name].start)
    solution = solver.find_another_solution(task_1._start)
print("Number of solutions:", nb_solution)

### Resource unavailability

In [ ]:
problem = SchedulingProblem("UnavailableResource", horizon=6)
nb_solution = 0
# only one task, there are many diffrent solutions
task_1 = FixedDurationTask("task1", duration=2)
worker_1 = Worker("Worker1")
task_1.add_required_resource(worker_1)
ResourceUnavailable(worker_1, [(0, 2), (4, 6)])

solver = SchedulingSolver(problem)
solution = solver.solve()

assert solution

solution.render_gantt_matplotlib(render_mode="Resource")